In [ ]:
from astropy.time import Time, TimeDelta
import astropy.units as u
from bokeh.models import DatetimeTickFormatter
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', logo=False)
from IPython.display import Markdown as md
import matplotlib.pyplot as plt
import numpy as np

from lsst_efd_client import EfdClient
from lsst.sitcom.mareuter import get_mplstyle_file

%matplotlib inline
plt.style.use(get_mplstyle_file())

In [ ]:
csc = "ATDomeTrajectory"
topic_name = "logevent_heartbeat"
expected_rate = 1
use_kafka = True
efd_name = "tucson_teststand_efd"
start_time_str = "2024-02-27T19:30:00"
end_time_str = "2024-02-27T20:10:00"

In [ ]:
client = EfdClient(efd_name)

In [ ]:
if ":" in csc:
    values = csc.split(":")
    csc_name = values[0]
    csc_index = int(values[1])
else:
    csc_name = csc
    csc_index = None
plot_title = f"{csc} {topic_name}"

In [ ]:
start_time = Time(start_time_str, scale='utc')
end_time = Time(end_time_str, scale='utc')

columns = ["private_sndStamp", "private_rcvStamp", "private_seqNum"]
if use_kafka:
    columns.append("private_kafkaStamp")
df = await client.select_time_series(f"lsst.sal.{csc_name}.{topic_name}",
                                     columns,
                                     start_time,
                                     end_time,
                                     index=csc_index)

In [ ]:
for column in columns:
    df[f"{column}_diff"] = df[column].diff()

In [ ]:
md(f"# <center>EFD Diagnostic Report<br><br>{plot_title}</center>")

In [ ]:
md(f"## <center>{start_time} - {end_time}</center>")

In [ ]:
md("### Breaks in Sequence Number")

In [ ]:
# Look for breaks in sequence numbers
seq_delta = df['private_seqNum_diff'].values
indexes = np.where(seq_delta > 1)[0]
if indexes.size:
    print("Break Size\tStart Timestamp\t\t\t\tEnd Timestamp")
    for index in indexes:
        #print(index)
        print(f"{seq_delta[index]}\t\t{df.index[index]}\t{df.index[index + 1]}")
        #print(df.index[index + 1] - df.index[index])
else:
    print("No breaks in sequence number found.")

In [ ]:
md("### Timing Information")

In [ ]:
delta2 = df["private_sndStamp_diff"].values
rate = 1 / expected_rate
message_send_jitter = (np.nanmedian(delta2) - rate) * u.s
if message_send_jitter < (1 * u.s):
    message_send_jitter = message_send_jitter.to(u.ms)
print(f"Median Message Send Jitter: {message_send_jitter:.4f}")

In [ ]:
min_t = np.round(np.min(delta2), decimals=3)
max_t = np.round(np.max(delta2), decimals=3)
nbins = 50
fig = plt.figure(1, (14, 6))
fig.suptitle("Distribution of $\Delta$T between Send Timestamps")
ax1 = fig.gca()
x = ax1.hist(delta2, bins=50, log=True)
ax1.set_xlabel("$\Delta$Snd (s)")
h = ax1.set_ylabel("Frequency")

In [ ]:
if use_kafka:
    deltak = df["private_kafkaStamp_diff"].values
    min_t = np.round(np.min(deltak), decimals=3)
    max_t = np.round(np.max(deltak), decimals=3)
    nbins = 50
    fig = plt.figure(1, (14, 6))
    fig.suptitle("Distribution of $\Delta$T between Kafka Timestamps")
    ax1 = fig.gca()
    x = ax1.hist(deltak, bins=50, log=True)
    ax1.set_xlabel("$\Delta$Kafka (s)")
    h = ax1.set_ylabel("Frequency")

In [ ]:
df.plot(y="private_kafkaStamp_diff", title="Time Evolution of ΔT Between Kafka Timestamps.",
        ylabel="ΔKafka (s)", backend="hvplot", width=1000, height=400)

In [ ]:
if not use_kafka:
    df["clock_diff"] = df["private_rcvStamp"] - df["private_sndStamp"]

In [ ]:
if not use_kafka:
    dt_format = '%F %T'
    formats = {'days': dt_format, 'months': dt_format, 'hours': dt_format, 'minutes': dt_format}
    date_formatter = DatetimeTickFormatter(**formats)
    tick_rotation = 75
    
    table = hv.Table(df.reset_index())
    x_tuple = ('index', 'clock_diff')
    clock_diff = hv.Curve(table, x_tuple, ("clock_diff")).opts(xlabel="Time", ylabel="Rcv - Snd (s)")
    layout = clock_diff
    layout.opts(opts.Curve(height=400, width=800, xformatter=date_formatter, xrotation=tick_rotation,
                           padding=0.01))

In [ ]:
if not use_kafka:
    max_t = np.round(np.max(df["clock_diff"].values), decimals=3)
    min_t = np.round(np.min(df["clock_diff"].values), decimals=3)
    if not min_t < 0:
        min_t = 0.
    nbins = 50
    fig = plt.figure(1, (14, 6))
    fig.suptitle("Distribution of Receive - Send Timestamps")
    ax1 = fig.gca()
    x = ax1.hist(df["clock_diff"].values, np.linspace(min_t, max_t, nbins), log=True)
    ax1.set_xlabel("Rcv - Snd (s)")
    h = ax1.set_ylabel("Frequency")